# Implementation Step (need to seperate to both models )

- Load dataset
- Load the model 
- Normalizing data
- Train and Capture the period of training time
- Save the model weight, log of training time and hyperparameter to text file
- 

In [ ]:
import torch





# Masure Step

- use the model to generate around 10000 sameples and masure FID score
- Visualize the score to compare 